In [ ]:
import matplotlib.pyplot as plt
import random
import time
import seaborn as sns
from datetime import datetime, timedelta
from IPython.display import display, clear_output
import warnings
from random import gauss
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from twilio.rest import Client

# Suppress FutureWarnings from Seaborn
warnings.filterwarnings("ignore", category=FutureWarning)

# Initialize empty lists to store data
time_values = []
heart_rate_values = []
temperature_values = []
blood_pressure_values = []

# Define mean (MU) and standard deviation (SD) values for vitals
HR_MU = 70
HR_SD = 20
SBP_MU = 120
SBP_SD = 25
TEMP_MU = 37
TEMP_SD = 0.5

# Flag to control sending alerts
send_alert = False

# Function to calculate stroke risk
def calculate_stroke_risk(heart_rate, temperature, blood_pressure):
    # Define risk factors based on provided logic
    heart_risk = 0
    temp_risk = 0
    bp_risk = 0
    
    # Heart Risk Logic
    if heart_rate <= 50:
        heart_risk += 1
    if heart_rate <= 40:
        heart_risk += 1
    if heart_rate >= 91:
        heart_risk += 1
    if heart_rate >= 110:
        heart_risk += 1
    if heart_rate >= 131:
        heart_risk += 1
    
    # Temperature Risk Logic
    if temperature <= 36:
        temp_risk += 1
    if temperature <= 35:
        temp_risk += 2
    if temperature >= 38.1:
        temp_risk += 1
    if temperature >= 39.1:
        temp_risk += 1
    
    # Blood Pressure Risk Logic
    if blood_pressure <= 110:
        bp_risk += 1
    if blood_pressure <= 100:
        bp_risk += 1
    if blood_pressure <= 90:
        bp_risk += 1
    if blood_pressure >= 220:
        bp_risk += 3
    
    # Calculate the total risk
    total_risk = heart_risk + temp_risk + bp_risk
    
    return total_risk

# Function to get random heart rate based on normal distribution
def get_random_heart_rate():
    return int(gauss(HR_MU, HR_SD))

# Function to get random systolic blood pressure based on normal distribution
def get_random_systolic_blood_pressure():
    return int(gauss(SBP_MU, SBP_SD))

# Function to get random temperature based on normal distribution
def get_random_temperature():
    return format(gauss(TEMP_MU, TEMP_SD), '.2f')

# Function to send an email
def send_email(subject, body):
    # Email configuration
    sender_email = 'tab495327@gmail.com'
    sender_password = 'gxdxbnfcsbkdvsdt'
    recipient_email = 'h83pratyush@gmail.com'

    # Create the email message
    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = recipient_email
    message['Subject'] = subject

    # Email body
    message.attach(MIMEText(body, 'plain'))

    # Connect to the SMTP server (in this example, we're using Gmail's SMTP server)
    try:
        smtp_server = smtplib.SMTP('smtp.gmail.com', 587)
        smtp_server.starttls()
        smtp_server.login(sender_email, sender_password)

        # Send the email
        smtp_server.sendmail(sender_email, recipient_email, message.as_string())

        # Close the SMTP server connection
        smtp_server.quit()
        
        print("Email sent successfully.")
    
    except Exception as e:
        print("Error sending email:", str(e))

# Function to send an SMS
def send_sms(message):
    # Twilio configuration
    twilio_account_sid = 'ACee30b71773459b75c35ee98d1b0b561e'
    twilio_auth_token = 'cc37f355a307530f4eae4a82e49db258'
    twilio_phone_number = '+18134457670'
    recipient_phone_number = '+919893958396'

    # Initialize Twilio client
    client = Client(twilio_account_sid, twilio_auth_token)

    try:
        # Send SMS
        message = client.messages.create(
            body=message,
            from_= +18134457670,
            to= +919893958396
        )

        print(f"SMS sent successfully. SID: {message.sid}")

    except Exception as e:
        print("Error sending SMS:", str(e))

# Create a function to update the real-time plot
def update_plot():
    plt.figure(figsize=(12, 8))
    
    # Set the background color to black
    plt.style.use('dark_background')
    
    # Calculate the time difference
    current_time = datetime.now()
    time_diff = [(current_time - t).total_seconds() for t in time_values]
    
    plt.subplot(411)
    sns.lineplot(x=time_diff, y=heart_rate_values, color='lime')
    plt.xlabel('Time')
    plt.ylabel('Heart Rate (BPM)', color='lime')
    plt.tick_params(axis='y', colors='lime')
    
    plt.subplot(412)
    sns.lineplot(x=time_diff, y=temperature_values, color='lightsteelblue')
    plt.xlabel('Time')
    plt.ylabel('Temperature (Celsius)', color='lightsteelblue')
    plt.tick_params(axis='y', colors='lightsteelblue')
    
    plt.subplot(413)
    sns.lineplot(x=time_diff, y=blood_pressure_values, color='yellow')
    plt.xlabel('Time')
    plt.ylabel('Systolic Blood Pressure (mmHg)', color='yellow')
    plt.tick_params(axis='y', colors='yellow')
    
    # Generate random vitals
    random_hr = get_random_heart_rate()
    random_temp = get_random_temperature()
    random_sbp = get_random_systolic_blood_pressure()
    
    # Append data to lists
    time_values.append(current_time)
    heart_rate_values.append(random_hr)
    temperature_values.append(random_temp)
    blood_pressure_values.append(random_sbp)
    
    # Keep data points for the last 20 seconds
    while time_values and (current_time - time_values[0]).total_seconds() > 20:
        time_values.pop(0)
        heart_rate_values.pop(0)
        temperature_values.pop(0)
        blood_pressure_values.pop(0)
    
    # Calculate stroke risk
    if time_values:
        risk_factor = calculate_stroke_risk(heart_rate_values[-1], float(temperature_values[-1]), blood_pressure_values[-1])
        risk_factor_displayed_at = current_time
    else:
        risk_factor_displayed_at = None

    # Create a subplot for displaying risk factor
    plt.subplot(414)
    if risk_factor_displayed_at:
        if risk_factor > 3:
            plt.text(0.5, 0.5, f"Risk Factor: {risk_factor}", color='red', fontsize=24, ha='center', va='center')
            plt.axis('off')  # Turn off axis
            plt.gca().set_facecolor('black')  # Set background color
            plt.tight_layout()
            clear_output(wait=True)
            display(plt.gcf())
            print("Alert: Stroke Risk Factor Exceeded 3")
            send_alert = True
            send_email("Stroke Risk Alert", "Alert: Stroke Risk Factor Exceeded 3")
            send_sms("Alert: Stroke Risk Factor Exceeded 3")  # Send SMS alert
        else:
            plt.text(0.5, 0.5, f"Risk Factor: {risk_factor}", color='green', fontsize=24, ha='center', va='center')
            plt.axis('off')  # Turn off axis
            plt.gca().set_facecolor('black')  # Set background color
    else:
        plt.text(0.5, 0.5, "Risk Factor: None", color='green', fontsize=24, ha='center', va='center')
        plt.axis('off')  # Turn off axis
        plt.gca().set_facecolor('black')  # Set background color
    
    # Customize the font size for the y-axis labels
    plt.yticks(fontsize=14)
    
    # Set the title and customize its appearance
    plt.suptitle('Vital Signs Monitor', color='white', fontsize=16)
    
    plt.tight_layout()
    clear_output(wait=True)
    display(plt.gcf())

# Set Seaborn style
sns.set_style("darkgrid")

try:
    while True:
        timestamp = datetime.now()
        
        if time_values:
            risk_factor = calculate_stroke_risk(heart_rate_values[-1], float(temperature_values[-1]), blood_pressure_values[-1])
            if risk_factor > 3 and not send_alert:
                # Risk factor is above 3, send an email and display an alert message
                send_email("Stroke Risk Alert", "Alert: Stroke Risk Factor Exceeded 3")
                send_sms("Alert: Stroke Risk Factor Exceeded 3")  # Send SMS alert
                send_alert = True
                print("Alert: Stroke Risk Factor Exceeded 3")
        
        update_plot()
        time.sleep(1)  # Update the plot every second
except KeyboardInterrupt:
    pass
